# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
# Import library yang dibutuhkan
import pandas as pd # Import library pandas untuk manipulasi data
from sklearn.model_selection import train_test_split # Import train_test_split untuk membagi data
from sklearn.preprocessing import StandardScaler # Import StandardScaler untuk normalisasi fitur
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier sebagai model klasifikasi
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix # Import metrics untuk evaluasi model

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Load dataset
# Membaca data dari file CSV
file_path = 'Dataset_inisiasi.csv'  # Pastikan file tersedia di direktori yang benar
df = pd.read_csv(file_path)


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:

# Data preprocessing
# Pastikan kolom 'cluster' bertipe integer dan hilangkan nilai NaN pada kolom tersebut
df['cluster'] = df['cluster'].astype(int)
df.dropna(subset=['cluster'], inplace=True)

# Encode kolom kategorikal menggunakan one-hot encoding
df = pd.get_dummies(df, columns=['provinsi', 'daerah', 'tahun_kategori'], drop_first=True)

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['cluster'])
y = df['cluster']

# Bagi data menjadi data latih dan data uji (80% latih, 20% uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [4]:
# Inisialisasi dan latih model RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [10]:

# Evaluasi model
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Cetak hasil evaluasi
print(f"Akurasi: {accuracy:.4f}")
print("\nLaporan Klasifikasi:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)

Akurasi: 0.9982

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       426
           1       1.00      1.00      1.00       411
           2       1.00      1.00      1.00       255

    accuracy                           1.00      1092
   macro avg       1.00      1.00      1.00      1092
weighted avg       1.00      1.00      1.00      1092


Confusion Matrix:
 [[426   0   0]
 [  2 409   0]
 [  0   0 255]]


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [ ]:
#Type your code here

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Type your code here

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Perbandingan Hasil Evaluasi.
  - Model menghasilkan akurasi sebesar 99.82%, yang menunjukkan performa yang sangat baik.
  - Dari laporan klasifikasi, precision, recall, dan f1-score semuanya mendekati 1.00, menandakan model mampu mengklasifikasikan data dengan sangat akurat.
Identifikasi Kelemahan Model
2. Identifikasi Kelemahan Model.
  - Dari confusion matrix, terdapat 2 kesalahan klasifikasi pada kelas 1, di mana dua sampel salah diklasifikasikan ke kelas lain.
  - Meskipun tingkat kesalahan sangat kecil, tetap perlu dipastikan apakah ada potensi overfitting, terutama jika akurasi data latih jauh lebih tinggi dari data uji.
  - Jika dataset memiliki distribusi yang tidak seimbang (jumlah sampel tiap kelas berbeda jauh), perlu dicek apakah model tetap generalizable untuk data baru.
3. Rekomendasi Tindakan Lanjutan.
  - Jika model terlalu sempurna pada dataset ini, bisa jadi ada overfitting. Coba uji dengan data baru atau validasi silang (cross-validation).
  - Untuk memastikan model tetap handal di dunia nyata, bisa dicoba menambahkan data baru atau menggunakan teknik augmentasi data jika memungkinkan.
  - Alternatif lain adalah mencoba model yang lebih sederhana, seperti mengurangi jumlah estimators pada Random Forest, untuk melihat apakah hasil tetap konsisten.
  - Jika model akan digunakan dalam skenario nyata, pertimbangkan untuk melakukan pengujian lebih lanjut dengan data real-world untuk melihat apakah akurasi tetap tinggi.